In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
%matplotlib inline

# Dan's ACF function

In [2]:
def dan_acf(x, axis=0, fast=False):
    """
    Estimate the autocorrelation function of a time series using the FFT.
    :param x:
        The time series. If multidimensional, set the time axis using the
        ``axis`` keyword argument and the function will be computed for every
        other axis.
    :param axis: (optional)
        The time axis of ``x``. Assumed to be the first axis if not specified.
    :param fast: (optional)
        If ``True``, only use the largest ``2^n`` entries for efficiency.
        (default: False)
    """
    x = np.atleast_1d(x)
    m = [slice(None), ] * len(x.shape)

    # For computational efficiency, crop the chain to the largest power of
    # two if requested.
    if fast:
        n = int(2**np.floor(np.log2(x.shape[axis])))
        m[axis] = slice(0, n)
        x = x
    else:
        n = x.shape[axis]

    # Compute the FFT and then (from that) the auto-correlation function.
    f = np.fft.fft(x-np.mean(x, axis=axis), n=2*n, axis=axis)
    m[axis] = slice(0, n)
    acf = np.fft.ifft(f * np.conjugate(f), axis=axis)[m].real
    m[axis] = 0
    return acf / acf[m]

In [ ]:
def corr_run(x, y, id, savedir, saveplot=True):

    gap_days = 0.02043365  # assume for long cadence
    jump_arr = scipy.array([131.51139, 169.51883, 169.75000, 182.00000, 200.31000,
                       231.00000, 246.19000, 256.00000, 260.22354, 281.00000,
                       291.00000, 322.00000, 352.37648, 373.23000, 384.00000,
                       398.00000, 443.48992, 475.50000, 504.00000, 539.44868,
                       567.00000, 599.00000, 630.17387, 661.00000, 691.00000,
                       711.20000, 735.36319, 762.00000, 808.51558, 845.00000,
                       874.50000, 906.84469, 937.00000, 970.00000, 1001.20718,
                       1032.50000, 1063.50000 ,1071.00000, 1093.60000])
    
    pylab.figure(1,(12, 9))
    pylab.clf()
    pylab.subplot(4,1,1)
    pylab.title('ID: %s' %(id_list[0]), fontsize = 16)
    pylab.axvspan(qt_max[j], qt_max[j+1], facecolor = 'k', alpha=0.1)
    pylab.plot(x, y, 'k-')
    for k in scipy.arange(len(jump_arr)):
        pylab.axvline(jump_arr[k], ls = '--', c = 'b')
    pylab.xlim(x.min(), x.max())
    pylab.ylim(min(y[scipy.isfinite(y) == True]), \
               max(y[scipy.isfinite(y) == True]))
    pylab.ylabel('Raw Flux')
    pylab.subplot(4,1,2)
    pylab.plot(x, y, 'k-')
    pylab.xlim(x.min(), x.max())
    pylab.ylim(x.min(), x.max())
    pylab.ylabel('Norm Flux')

    # max period searched for is len(flux) / 2
    max_psearch_len = len(y) / 2.0

    acf_tab, acf_per_pos, acf_per_height, acf_per_err, locheight, asym,  = \
        acf_calc(time=x, flux=y, interval=gap_days, kid = id, max_psearch_len = max_psearch_len)

    pylab.figure(1)
    pylab.subplot(4,1,4)
    pylab.plot(acf_tab.lags_days, acf_tab.acf_smooth, 'k-')
    pylab.axhline(0, ls = '--', c = 'k')
    for i in scipy.arange(len(acf_per_pos)):
        pylab.axvline(acf_per_pos[i], ls = '--', c = 'b')
    pylab.ylabel('ACF')
    pylab.xlim(0, acf_tab.lags_days.max())

    pylab.figure(12)
    pylab.clf()
    pylab.plot(acf_tab.lags_days, acf_tab.acf_smooth, 'k-')
    for m in scipy.arange(len(acf_per_pos)):
        pylab.axvline(acf_per_pos[m], ls = '--', c = 'r')

    # plot and calculate acf peak statistics
    if acf_per_pos[0] != -9999:
        med_dlag_per[x], dlag_per_err[x], acf_peak_per[x], h1[x], w1[x], lh1[x], \
            hlocgrad[x], hloc_grad_scatter[x], width_grad[x], width_grad_scatter[x], \
            num_of_peaks[x], harmonic_det[x], sel_peaks, one_peak_only, peak_ratio =\
            plot_stats(lc_tab.time, lc_tab.flux, x, acf_per_pos, \
                       acf_per_height, acf_per_err, locheight, asym)

        n_s = 'k'

        period[x] = acf_peak_per[x]
#         print('PEAK RATIO = ', peak_ratio)

        # plot period lines on full plot
        pylab.figure(1)
        pylab.subplot(4,1,4)
        if med_dlag_per[x] >0:
            for n in scipy.arange(len(sel_peaks)):
                pylab.axvline(sel_peaks[n], ls = '--', c = 'r')
        #pylab.axvline(period[x], ls = '--', c = 'k')
        pylab.axvline(period[x], ls = '--', c = n_s)
        if med_dlag_per[x] > 0:
            pylab.axvspan(med_dlag_per[x]-dlag_per_err[x], \
                          med_dlag_per[x]+dlag_per_err[x],\
                facecolor = 'k', alpha=0.2)
        pylab.xlim(period[x] * 10)

        # variability stats
        print('calculating var for P_med...')
        amp_all[x], amp_per[x], per_cent, var_arr_real = \
            calc_var(kid = x, time_in = lc_tab.time, \
                     flux = lc_tab.flux, period = acf_peak_per[x])

        pylab.figure(1)
        pylab.subplot(4,1,3)
        pylab.plot(per_cent, var_arr_real, 'k.')
        pylab.axhline(amp_per[x], ls = '--', c = 'b')
        pylab.xlim(lc_tab.time.min(),lc_tab.time.max())
        pylab.ylim(var_arr_real.min(), var_arr_real.max())
        ax = pylab.gca()
        pylab.text(0.415, -0.15, 'Time (days)', transform = ax.transAxes)
        pylab.text(0.415, -1.4, 'Period (days)', transform = ax.transAxes)
        pylab.ylabel('Amplitudes')
        pylab.xlim(period[x] * 10)

        if saveplot:
            print("saving figure", "%s/%s_full.png" % (savedir, id_list[0]))
            pylab.savefig('%s/%s_full.png' %(savedir, id_list[0]))

        maxpts = 40.0
        if scipy.floor(lc_tab.time.max() / acf_peak_per[x]) < maxpts:
            maxpts = float(scipy.floor(lc_tab.time.max() / acf_peak_per[x]))
        inc = lc_tab.time - lc_tab.time.min() <= (maxpts*acf_peak_per[x])

#         print('**************************', 'KID = ', x, 'PEAK HEIGHT = ', \
#             max(acf_per_height[:2]), 'LOCAL PEAK HEIGHT = ', lh1[x])

        t_stats = atpy.Table()
        t_stats.add_column('acf_per_pos', acf_per_pos)
        t_stats.add_column('acf_per_height', acf_per_height)
        t_stats.add_column('acf_per_err', acf_per_err)
        t_stats.add_column('asym', asym)
        t_stats.add_column('locheight', locheight)

        if dlag_per_err[x] == 0.:
             error = acf_per_err[x]
        else: error = dlag_per_err[x]

        print('PERIOD = ', period[x], '+/-', error)
        print('saving as', '%s/%s_result.txt'%(savedir, id_list[0]))
        np.savetxt('%s/%s_result.txt'%(savedir, id_list[0]),
                   np.transpose((period[x], error)))
    else:
        blank = np.array([0,0])
#         np.savetxt('%s/%s_result.txt' %(savedir, id_list[0]), blank)

    t = atpy.Table()
    t.add_column('period', period) #period
    t.add_column('sine_per', sine_per) #sine period
    t.add_column('sine_height', sine_height)
    t.add_column('acf_peak_per', acf_peak_per)
    t.add_column('med_dlag_per', med_dlag_per)
    t.add_column('dlag_per_err', dlag_per_err) #error
    t.add_column('h1', h1)
    t.add_column('w1', w1)
    t.add_column('lh1', lh1)
    t.add_column('hlocgrad', hlocgrad)
    t.add_column('hloc_grad_scatter', hloc_grad_scatter)
    t.add_column('width_grad', width_grad)
    t.add_column('width_grad_scatter', width_grad_scatter)
    t.add_column('num_of_peaks', num_of_peaks)
    t.add_column('harmonic_det', harmonic_det)
    t.add_column('amp_all', amp_all)
    t.add_column('amp_per', amp_per)
    return period, dlag_per_err


# ACF calc function

In [ ]:
def acf_calc(time, flux, interval, kid, max_psearch_len):

    ''' Calculate ACF, calls error calc function'''

    # ACF calculation in pylab, close fig when finished
    pylab.figure(50)
    pylab.clf()
    lags, acf, lines, axis = pylab.acorr(flux, maxlags = max_psearch_len)
    pylab.close(50)
    #acf = dan_acf(flux)

    #convolve smoothing window with Gaussian kernel
    gauss_func = lambda x,sig: 1./np.sqrt(2*np.pi*sig**2) * \
                 np.exp(-0.5*(x**2)/(sig**2)) #define a Gaussian
    #create the smoothing kernel
    conv_func = gauss_func(np.arange(-28,28,1.),9.)

    acf_smooth = np.convolve(acf,conv_func,mode='same') #and convolve
    lenlag = len(lags)
    lags = lags[int(lenlag/2.0):lenlag][:-1] * interval
    acf = acf[int(lenlag/2.0): lenlag][0:-1]
    acf_smooth = acf_smooth[int(lenlag/2.0): lenlag][1:]

    # find max using usmoothed acf (for plot only)
    max_ind_us, max_val_us = extrema(acf, max = True, min = False)

    # find max/min using smoothed acf
    max_ind_s, max_val_s = extrema(acf_smooth, max = True, min = False)
    min_ind_s, min_val_s = extrema(acf_smooth, max = False, min = True)
    maxmin_ind_s, maxmin_val_s = extrema(acf_smooth, max = True, min = True)

    if len(max_ind_s) > 0 and len(min_ind_s) > 0:
        # ensure no duplicate peaks are detected
        t_max_s = atpy.Table()
        t_max_s.add_column('ind', max_ind_s)
        t_max_s.add_column('val', max_val_s)
        t_min_s = atpy.Table()
        t_min_s.add_column('ind', min_ind_s)
        t_min_s.add_column('val', min_val_s)
        t_maxmin_s = atpy.Table()
        t_maxmin_s.add_column('ind', maxmin_ind_s)
        t_maxmin_s.add_column('val', maxmin_val_s)

        ma_i = collections.Counter(t_max_s.ind)
        dup_arr = [i for i in ma_i if ma_i[i]>1]
        if len(dup_arr) > 0:
            for j in scipy.arange(len(dup_arr)):
                tin = t_max_s.where(t_max_s.ind != dup_arr[j])
                tout = t_max_s.where(t_max_s.ind == dup_arr[j])
                tout = tout.rows([0])
                tin.append(tout)
            t_max_s = copy.deepcopy(tin)

        ma_i = collections.Counter(t_min_s.ind)
        dup_arr = [i for i in ma_i if ma_i[i]>1]
        if len(dup_arr) > 0:
            for j in scipy.arange(len(dup_arr)):
                tin = t_min_s.where(t_min_s.ind != dup_arr[j])
                tout = t_min_s.where(t_min_s.ind == dup_arr[j])
                tout = tout.rows([0])
                tin.append(tout)
            t_min_s = copy.deepcopy(tin)

        ma_i = collections.Counter(t_maxmin_s.ind)
        dup_arr = [i for i in ma_i if ma_i[i]>1]
        if len(dup_arr) > 0:
            for j in scipy.arange(len(dup_arr)):
                tin = t_maxmin_s.where(t_maxmin_s.ind != dup_arr[j])
                tout = t_maxmin_s.where(t_maxmin_s.ind == dup_arr[j])
                tout = tout.rows([0])
                tin.append(tout)
            t_maxmin_s = copy.deepcopy(tin)

        t_max_s.sort('ind')
        t_min_s.sort('ind')
        t_maxmin_s.sort('ind')

        # relate max inds to lags
        maxnum = len(t_max_s.ind)
        acf_per_pos = lags[t_max_s.ind]
        acf_per_height = acf[t_max_s.ind]

        print('Calculating errors and asymmetries...')
        # Calculate peak widths, asymmetries etc
        acf_per_err, locheight, asym= \
            calc_err(kid = kid, lags = lags, acf = acf, inds = \
            t_maxmin_s.ind, vals = t_maxmin_s.val, maxnum = maxnum)

    else:
        acf_per_pos = scipy.array([-9999])
        acf_per_height = scipy.array([-9999])
        acf_per_err = scipy.array([-9999])
        locheight = scipy.array([-9999])
        asym = scipy.array([-9999])

    # save corrected LC and ACF
    t_lc = atpy.Table()
    t_lc.add_column('time', time)
    t_lc.add_column('flux', flux)

    t_acf = atpy.Table()
    t_acf.add_column('lags_days', lags)
    t_acf.add_column('acf', acf)
    t_acf.add_column('acf_smooth', acf_smooth)

    pylab.figure(6,(10, 5.5))
    pylab.clf()
    pylab.plot(t_acf.lags_days, t_acf.acf, 'k-')
    pylab.plot(t_acf.lags_days, t_acf.acf_smooth, 'r-')
    for j in scipy.arange(len(max_ind_us)):
        pylab.axvline(t_acf.lags_days[max_ind_us[j]], ls = '--', c = 'k', lw=1)
    for i in scipy.arange(len(acf_per_pos)):
        pylab.axvline(acf_per_pos[i], ls = '--', c = 'r', lw = 2)
    if t_acf.lags_days.max() > 10 * acf_per_pos[0]:
        pylab.xlim(0,10 * acf_per_pos[0])
    else: pylab.xlim(0,max(t_acf.lags_days))
    pylab.xlabel('Period (days)')
    pylab.ylabel('ACF')
    pylab.title('ID: %s, Smoothed \& Unsmoothed ACF' %(kid))
    return  t_acf, acf_per_pos, acf_per_height, acf_per_err, locheight, asym